In [1]:
import os
import shutil
from pathlib import Path
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import matplotlib.pyplot as plt
import itertools
     

In [2]:
def copy_images(src_folder, dst_folder, max_images=10000):
    count = 0
    files = sorted(os.listdir(src_folder))
    for f in files:
        src_path = os.path.join(src_folder, f)
        if os.path.isfile(src_path) and os.path.splitext(f)[1].lower() in valid_exts:
            shutil.copy(src_path, os.path.join(dst_folder, f))
            count += 1
            if count >= max_images:
                break
    return count

In [3]:
class ImageDataset(Dataset):
    def __init__(self, root, transform=None):
        self.transform = transform
        self.files_A = sorted([p for p in Path(root, "trainA").glob("*") if p.suffix.lower() in valid_exts])
        self.files_B = sorted([p for p in Path(root, "trainB").glob("*") if p.suffix.lower() in valid_exts])

    def __len__(self):
        return min(len(self.files_A), len(self.files_B))

    def __getitem__(self, idx):
        img_A = Image.open(self.files_A[idx]).convert("RGB")
        img_B = Image.open(self.files_B[idx]).convert("RGB")

        if self.transform:
            img_A = self.transform(img_A)
            img_B = self.transform(img_B)

        return {"A": img_A, "B": img_B}
     

In [4]:

class ResnetBlock(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(dim, dim, 3, 1, 1),
            nn.InstanceNorm2d(dim),
            nn.ReLU(inplace=True),
            nn.Conv2d(dim, dim, 3, 1, 1),
            nn.InstanceNorm2d(dim),
        )
    def forward(self, x):
        return x + self.block(x)

In [5]:
class Generator(nn.Module):
    def __init__(self, in_nc, out_nc, n_res_blocks=6):
        super().__init__()
        layers = [
            nn.Conv2d(in_nc, 64, 7, 1, 3),
            nn.InstanceNorm2d(64),
            nn.ReLU(inplace=True),

            nn.Conv2d(64, 128, 3, 2, 1),
            nn.InstanceNorm2d(128),
            nn.ReLU(inplace=True),

            nn.Conv2d(128, 256, 3, 2, 1),
            nn.InstanceNorm2d(256),
            nn.ReLU(inplace=True),
        ]

        for _ in range(n_res_blocks):
            layers.append(ResnetBlock(256))

        layers += [
            nn.ConvTranspose2d(256, 128, 3, 2, 1, output_padding=1),
            nn.InstanceNorm2d(128),
            nn.ReLU(inplace=True),

            nn.ConvTranspose2d(128, 64, 3, 2, 1, output_padding=1),
            nn.InstanceNorm2d(64),
            nn.ReLU(inplace=True),

            nn.Conv2d(64, out_nc, 7, 1, 3),
            nn.Tanh(),
        ]

        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

In [6]:
class Discriminator(nn.Module):
    def __init__(self, in_nc):
        super().__init__()
        layers = [
            nn.Conv2d(in_nc, 64, 4, 2, 1),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(64, 128, 4, 2, 1),
            nn.InstanceNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(128, 256, 4, 2, 1),
            nn.InstanceNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(256, 512, 4, 1, 1),
            nn.InstanceNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(512, 1, 4, 1, 1),
        ]
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

In [7]:
def denorm(x):
    return x * 0.5 + 0.5

In [8]:
celeba_img_folder = r"D:\sangita-mam\assginment-13\img_align_celeba\img_align_celeba"
portrait_img_folder = r"C:\Users\VRLAB02\OneDrive\Desktop\data\paintings"

print("Checking if CelebA folder exists:", os.path.exists(celeba_img_folder))
print("Checking if Portrait folder exists:", os.path.exists(portrait_img_folder))


root_dir = "cyclegan_faces"
trainA_dir = os.path.join(root_dir, "trainA")
trainB_dir = os.path.join(root_dir, "trainB")
os.makedirs(trainA_dir, exist_ok=True)
os.makedirs(trainB_dir, exist_ok=True)

valid_exts = [".jpg", ".jpeg", ".png"]
numA = copy_images(celeba_img_folder, trainA_dir, 3144)
numB = copy_images(portrait_img_folder, trainB_dir, 3144)

print(f"Copied {numA} images to {trainA_dir}")
print(f"Copied {numB} images to {trainB_dir}")

print("Final counts:")
print("trainA images:", len(os.listdir(trainA_dir)))
print("trainB images:", len(os.listdir(trainB_dir)))

# Define Dataset class
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3),
])



dataset = ImageDataset(root_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True, num_workers=0)


print(f"Dataset length: {len(dataset)}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
G_AB = Generator(3, 3).to(device)
G_BA = Generator(3, 3).to(device)
D_A = Discriminator(3).to(device)
D_B = Discriminator(3).to(device)

criterion_GAN = nn.MSELoss()
criterion_cycle = nn.L1Loss()

optim_G = optim.Adam(itertools.chain(G_AB.parameters(), G_BA.parameters()), lr=2e-4, betas=(0.5, 0.999))
optim_D_A = optim.Adam(D_A.parameters(), lr=2e-4, betas=(0.5, 0.999))
optim_D_B = optim.Adam(D_B.parameters(), lr=2e-4, betas=(0.5, 0.999))



os.makedirs("outputs", exist_ok=True)

num_epochs = 25
for epoch in range(1, num_epochs + 1):
    for batch in dataloader:
        real_A = batch["A"].to(device)
        real_B = batch["B"].to(device)

        # Train Generators
        optim_G.zero_grad()

        fake_B = G_AB(real_A)
        fake_A = G_BA(real_B)

        loss_GAN_AB = criterion_GAN(D_B(fake_B), torch.ones_like(D_B(fake_B)))
        loss_GAN_BA = criterion_GAN(D_A(fake_A), torch.ones_like(D_A(fake_A)))

        recov_A = G_BA(fake_B)
        recov_B = G_AB(fake_A)

        loss_cycle_A = criterion_cycle(recov_A, real_A)
        loss_cycle_B = criterion_cycle(recov_B, real_B)

        loss_G = loss_GAN_AB + loss_GAN_BA + 10 * (loss_cycle_A + loss_cycle_B)
        loss_G.backward()
        optim_G.step()

        # Train Discriminator A
        optim_D_A.zero_grad()
        loss_D_A = 0.5 * (criterion_GAN(D_A(real_A), torch.ones_like(D_A(real_A))) +
                          criterion_GAN(D_A(fake_A.detach()), torch.zeros_like(D_A(fake_A))))
        loss_D_A.backward()
        optim_D_A.step()

        # Train Discriminator B
        optim_D_B.zero_grad()
        loss_D_B = 0.5 * (criterion_GAN(D_B(real_B), torch.ones_like(D_B(real_B))) +
                          criterion_GAN(D_B(fake_B.detach()), torch.zeros_like(D_B(fake_B))))
        loss_D_B.backward()
        optim_D_B.step()

    print(f"Epoch {epoch} | G Loss: {loss_G.item():.4f} | D_A Loss: {loss_D_A.item():.4f} | D_B Loss: {loss_D_B.item():.4f}")

    if epoch % 5 == 0:
        G_AB.eval()
        with torch.no_grad():
            # Get 5 real images from dataset
            real = torch.stack([dataset[i]["A"] for i in range(5)], dim=0).to(device)
            fake = G_AB(real).cpu()
            real = real.cpu()

            real = denorm(real)
            fake = denorm(fake)

            # Create two rows: one for real, one for fake
            real_row = utils.make_grid(real, nrow=5, padding=2)
            fake_row = utils.make_grid(fake, nrow=5, padding=2)

            # Stack vertically (CHW)
            grid = torch.cat([real_row, fake_row], dim=1)  # concat in height

            # Plot
            plt.figure(figsize=(12, 5))
            plt.imshow(grid.permute(1, 2, 0))
            plt.axis('off')
            plt.title(f'CycleGAN Output - Epoch {epoch}')
            plt.savefig(f"outputs/cyclegan_epoch_{epoch}.png")
            plt.close()
        G_AB.train()


Checking if CelebA folder exists: True
Checking if Portrait folder exists: True
Copied 3144 images to cyclegan_faces\trainA
Copied 3144 images to cyclegan_faces\trainB
Final counts:
trainA images: 3144
trainB images: 3144
Dataset length: 3144
cuda
Epoch 1 | G Loss: 5.3666 | D_A Loss: 0.1649 | D_B Loss: 0.0441
Epoch 2 | G Loss: 5.6223 | D_A Loss: 0.0664 | D_B Loss: 0.1780
Epoch 3 | G Loss: 3.7785 | D_A Loss: 0.1074 | D_B Loss: 0.2073
Epoch 4 | G Loss: 3.6984 | D_A Loss: 0.1075 | D_B Loss: 0.0502
Epoch 5 | G Loss: 3.9348 | D_A Loss: 0.1659 | D_B Loss: 0.2188
Epoch 6 | G Loss: 4.6372 | D_A Loss: 0.0601 | D_B Loss: 0.1366
Epoch 7 | G Loss: 3.1381 | D_A Loss: 0.1280 | D_B Loss: 0.1280
Epoch 8 | G Loss: 2.7244 | D_A Loss: 0.1443 | D_B Loss: 0.2261
Epoch 9 | G Loss: 4.1183 | D_A Loss: 0.1160 | D_B Loss: 0.2146
Epoch 10 | G Loss: 3.6623 | D_A Loss: 0.2489 | D_B Loss: 0.2617
Epoch 11 | G Loss: 3.5102 | D_A Loss: 0.0424 | D_B Loss: 0.0742
Epoch 12 | G Loss: 2.8221 | D_A Loss: 0.0577 | D_B Loss: 